In [1]:
from mxnet.gluon import nn
from mxnet import nd

def get_net():
    net = nn.Sequential()
    with net.name_scope():
        net.add(nn.Dense(4,activation="relu"))
        net.add(nn.Dense(2))
    return net

x = nd.random.uniform(shape=(3,5))

不call initialize()

In [2]:
import sys
try:
    net = get_net()
    net(x)
except RuntimeError as err:
    sys.stderr.write(str(err))

Parameter 'sequential0_dense0_weight' has not been initialized. Note that you should initialize parameters and create Trainer with Block.collect_params() instead of Block.params because the later does not include Parameters of nested child Blocks

In [3]:
net.initialize()
net(x)


[[0.00212593 0.00365805]
 [0.00161272 0.00441845]
 [0.00204872 0.00352518]]
<NDArray 3x2 @cpu(0)>

访问模型参数

In [4]:
W = net[0].weight
b = net[0].bias
print('name:',net[0].name,'\nweight:',W,'\nbias:',b)

name: sequential0_dense0 
weight: Parameter sequential0_dense0_weight (shape=(4, 5), dtype=float32) 
bias: Parameter sequential0_dense0_bias (shape=(4,), dtype=float32)


In [5]:
print('weight',W.data())
print('weight gradient',W.grad())
print('bias',b.data())
print('bias gradient',b.grad())

weight 
[[-0.06206018  0.06491279 -0.03182812 -0.01631819 -0.00312688]
 [ 0.0408415   0.04370362  0.00404529 -0.0028032   0.00952624]
 [-0.01501013  0.05958354  0.04705103 -0.06005495 -0.02276454]
 [-0.0578019   0.02074406 -0.06716943 -0.01844618  0.04656678]]
<NDArray 4x5 @cpu(0)>
weight gradient 
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
<NDArray 4x5 @cpu(0)>
bias 
[0. 0. 0. 0.]
<NDArray 4 @cpu(0)>
bias gradient 
[0. 0. 0. 0.]
<NDArray 4 @cpu(0)>


In [6]:
params = net.collect_params()
print(params)
print(params['sequential0_dense0_bias'].data())
print(params.get('dense0_weight').data())

sequential0_ (
  Parameter sequential0_dense0_weight (shape=(4, 5), dtype=float32)
  Parameter sequential0_dense0_bias (shape=(4,), dtype=float32)
  Parameter sequential0_dense1_weight (shape=(2, 4), dtype=float32)
  Parameter sequential0_dense1_bias (shape=(2,), dtype=float32)
)

[0. 0. 0. 0.]
<NDArray 4 @cpu(0)>

[[-0.06206018  0.06491279 -0.03182812 -0.01631819 -0.00312688]
 [ 0.0408415   0.04370362  0.00404529 -0.0028032   0.00952624]
 [-0.01501013  0.05958354  0.04705103 -0.06005495 -0.02276454]
 [-0.0578019   0.02074406 -0.06716943 -0.01844618  0.04656678]]
<NDArray 4x5 @cpu(0)>


In [7]:
from mxnet import init
params.initialize(init=init.Normal(sigma=0.02),force_reinit=True)
print(net[0].weight.data(),net[0].bias.data())


[[ 0.02804598  0.00220872  0.00701151  0.02721515  0.00500832]
 [ 0.00112992  0.03227538 -0.01813176 -0.00385197 -0.01286032]
 [ 0.03360647 -0.02855298 -0.03083278 -0.02110904 -0.02623654]
 [-0.00293494  0.01282986 -0.01476416  0.04062728  0.01186533]]
<NDArray 4x5 @cpu(0)> 
[0. 0. 0. 0.]
<NDArray 4 @cpu(0)>


In [8]:
params.initialize(init=init.One(),force_reinit=True)
print(net[0].weight.data(),net[0].bias.data())


[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
<NDArray 4x5 @cpu(0)> 
[0. 0. 0. 0.]
<NDArray 4 @cpu(0)>


In [17]:
class MyInit(init.Initializer):
    def __init__(self):
        super(MyInit,self).__init__()
        self._verbose = True
    def _init_weight(self,_,arr):
        print('init weight',arr.shape)
        nd.random.uniform(low=5,high=10,out=arr)
    def _init_bias(self,_,arr):
        pritn('init bias',arr.shape)
        arr[:] = 2

params.initialize(init=MyInit(),force_reinit=True)
print(net[0].weight.data(),net[0].bias.data())

init weight (4, 5)
init weight (2, 4)

[[9.205701  5.7472415 9.523239  9.34063   5.187797 ]
 [5.8124647 7.5415773 8.077798  5.8342376 5.6191   ]
 [8.895255  9.240042  9.324667  9.036594  7.0569835]
 [7.845504  5.699863  7.0359163 5.166112  5.3458347]]
<NDArray 4x5 @cpu(0)> 
[0. 0. 0. 0.]
<NDArray 4 @cpu(0)>


In [ ]:
net = get_net()
print(net.collect_params())